In [303]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm

In [304]:
traindf = pd.read_csv("train.csv")
traindf_dropna = traindf.dropna()
traindf_complete = traindf_dropna.loc[:,['ora',
                                         'irr_pvgis_mod','irri_pvgis_mod',
                                         'temperatura_ambiente', 'tempi',
                                         'irradiamento','irri',
                                         'pressure','pressurei', 'windspeed','windspeedi',
                                         'dewpoint','dewpointi', 'cloudcover','cloudcoveri', 'kwh']]


# traindf_complete = traindf_dropna.loc[:,['ora',
#                                          'irradiamento','irri','cloudcover','cloudcoveri',
#                                          'kwh']]

X = traindf_complete.drop(['kwh'], axis=1)
Y = traindf_complete['kwh']

X.head(3)


,ora,irr_pvgis_mod,irri_pvgis_mod,temperatura_ambiente,tempi,irradiamento,irri,pressure,pressurei,windspeed,windspeedi,dewpoint,dewpointi,cloudcover,cloudcoveri
0,0.0,0.0,0.189,0.212,0.113,0.0,0.219,0.0,0.0,0.054,0.040,0.436,0.114,0.0,0.200
1,0.0,0.0,0.200,0.212,0.117,0.0,0.220,0.0,0.0,0.062,0.038,0.444,0.188,0.0,0.329
2,0.0,0.0,0.158,0.212,0.115,0.0,0.220,0.0,0.0,0.054,0.039,0.432,0.201,0.0,0.201


In [305]:
# x_train,x_test,y_train,y_test = train_test_split(X,Y, test_size=0.4)

# y_test.head(10)

In [306]:
# clf = svm.SVR(C=1, gamma=10, epsilon=0.1).fit(x_train, y_train)
# clf.score(x_test, y_test)

In [ ]:
# clf.predict(x_test)[:10]

In [ ]:
#####################################################################

In [ ]:

testdf = pd.read_csv("test.csv")
# testdf_dropna = testdf.dropna()
testdf_complete = testdf.loc[:,['ora',
                                         'irr_pvgis_mod','irri_pvgis_mod',
                                         'temperatura_ambiente', 'tempi',
                                         'irradiamento','irri',
                                         'pressure','pressurei', 'windspeed','windspeedi',
                                         'dewpoint','dewpointi', 'cloudcover','cloudcoveri']]

# testdf_complete = testdf_dropna.loc[:,['ora',
#                                          'irradiamento','irri','cloudcover','cloudcoveri'
#                                       ]]

testdf_complete.head(3)

In [ ]:
modelfit = svm.SVR(C=100, gamma=0.1, epsilon=0.1).fit(X, Y)
print('ok')

In [ ]:
pred_result = modelfit.predict(testdf_complete)

In [ ]:
# pred_result[140:150]

In [ ]:
# testdf_complete[140:150]

In [ ]:
kwh = pd.Series(pred_result)
d = {'id': testdf['id'].tolist(), 'kwh': pred_result}
result = pd.DataFrame(data=d)
result.head()

In [ ]:
result.to_csv('wynik.csv',index=False)